In [1]:
import tensorflow as tf
import numpy as np

In [2]:
sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))

W0924 23:24:43.690877  4112 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0924 23:24:43.693877  4112 deprecation.py:323] From <ipython-input-2-84b3bfe2f492>:21: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0924 23:24:43.718862  4112 deprecation.py:323] From <ipython-input-2-84b3bfe2f492>:23: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`

0 loss: 2.2920775 Prediction: y    u  uuuuuuu
1 loss: 2.1139 Prediction: y         u    
2 loss: 1.9401435 Prediction: y   ou    u  ou
3 loss: 1.7268242 Prediction: yy you   ot you
4 loss: 1.464325 Prediction: y  you   nt you
5 loss: 1.1279991 Prediction: y  you wnnt you
6 loss: 0.85951537 Prediction: yy you want you
7 loss: 0.6370599 Prediction: iy you want you
8 loss: 0.4526128 Prediction: iy you want you
9 loss: 0.31169203 Prediction: if you want you
10 loss: 0.21148446 Prediction: if you want you
11 loss: 0.14163305 Prediction: if you want you
12 loss: 0.095184155 Prediction: if you want you
13 loss: 0.0648531 Prediction: if you want you
14 loss: 0.04444238 Prediction: if you want you
15 loss: 0.030596359 Prediction: if you want you
16 loss: 0.021349855 Prediction: if you want you
17 loss: 0.015213315 Prediction: if you want you
18 loss: 0.011101429 Prediction: if you want you
19 loss: 0.00829849 Prediction: if you want you
20 loss: 0.0063514947 Prediction: if you want you
21 loss: